In [239]:
def whatsthismer(id):
    
    return raw[raw['merchantid']==id]

In [684]:
import pandas as pd
from datetime import datetime

In [786]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702979 entries, 0 to 702978
Data columns (total 31 columns):
status              702979 non-null object
notifytime          702979 non-null object
amount              702979 non-null int64
client              702979 non-null object
merchantid          702979 non-null int64
merchantname        702979 non-null object
channel             702979 non-null object
settlementperiod    702979 non-null object
scantype            659006 non-null object
cardtype            702979 non-null object
qrcode              219772 non-null object
opentime            702912 non-null float64
merchanttype        702912 non-null object
paymentidentity     106654 non-null object
bankcardnum         85982 non-null object
paymentchannel      106664 non-null object
terminalid          702976 non-null object
codeoffail          10228 non-null object
causeoffail         10225 non-null object
aa.date             702979 non-null int64
isIntAmount         702979 non-nul

In [484]:
raw=pd.read_csv('data/raw01',encoding='utf-8', sep='\t', low_memory=False)
raw.drop(['checkpoint'],axis=1,inplace=True)

In [485]:
# # 对原始样本数据进行加工
# ali=pd.read_csv('data/alirisk.csv',encoding='ANSI')
# we=pd.read_csv('data/wechatrisk.csv',encoding='ANSI')
# ali.columns=['alimid','alidate','alitype','alidecription']
# ali.head(10)
# ali.to_csv('ali001.csv',index=False,encoding='utf_8_sig')
# we.columns=['wemname','wedate','wemethod','wetype','wedescription']
# we.head(10)
# we.to_csv('we001.csv',index=False,encoding='utf_8_sig')

## 对标签进行加工，仅保留单一处罚类型，例如多次同类型处罚保留最新一次，多次不同类型处罚视情况保留其中一类
ali=pd.read_csv('data/alirisk001.csv',encoding='ANSI')
we=pd.read_csv('data/wechatrisk001.csv',encoding='ANSI')

raw=pd.merge(raw,ali,how='left',left_on=['merchantid'],right_on=['alimid'])
raw=pd.merge(raw,we,how='left',left_on=['merchantname'],right_on=['wemname'])

risk=raw[pd.notnull(raw['alitype'])][['merchantid','merchantname']]\
.append(raw[pd.notnull(raw['wetype'])][['merchantid','merchantname']], ignore_index=True)\
.drop_duplicates()\
.reset_index(drop=True)

In [782]:
raw[(pd.notnull(raw['alitype'])) | (pd.notnull(raw['wetype']))].columns

Index(['status', 'notifytime', 'amount', 'client', 'merchantid',
       'merchantname', 'channel', 'settlementperiod', 'scantype', 'cardtype',
       'qrcode', 'opentime', 'merchanttype', 'paymentidentity', 'bankcardnum',
       'paymentchannel', 'terminalid', 'codeoffail', 'causeoffail', 'aa.date',
       'isIntAmount', 'isNightAmount', 'alimid', 'alidate', 'alitype',
       'alidecription', 'wemname', 'wedate', 'wemethod', 'wetype',
       'wedescription'],
      dtype='object')

In [784]:
raw[(pd.notnull(raw['alitype'])) | (pd.notnull(raw['wetype']))][['merchantid','alitype', 'alidecription', 'wemname', 'wedate',
                                                                 'wemethod', 'wetype', 'wedescription']].to_csv('data/risk.csv',encoding='utf_8_sig')

In [706]:
raw.columns

Index(['status', 'notifytime', 'amount', 'client', 'merchantid',
       'merchantname', 'channel', 'settlementperiod', 'scantype', 'cardtype',
       'qrcode', 'opentime', 'merchanttype', 'paymentidentity', 'bankcardnum',
       'paymentchannel', 'terminalid', 'codeoffail', 'causeoffail', 'aa.date',
       'isIntAmount', 'isNightAmount', 'alimid', 'alidate', 'alitype',
       'alidecription', 'wemname', 'wedate', 'wemethod', 'wetype',
       'wedescription'],
      dtype='object')

## 近7天/近14天 发起交易完成交易
### 交易相关
交易笔数 金额 均值 标准差 最大值 最小值 中位数
云闪付信用卡交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
微信信用卡交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数（支付宝没有卡类型信息）
整数交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
大额交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
夜间交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
同额度交易笔数 占比 金额 占比
交易频繁程度（平均间隔时间）
交易失败率

### 卡相关
单卡交易笔数 占比 最大值 金额 占比 最大值
卡号交易比（卡数量比交易量）

### 用户相关
单用户交易笔数 占比 最大值 金额 占比 最大值
用户交易比（用户数量比交易量）

In [486]:
# 判断整数交易
raw['isIntAmount']=raw['amount'].apply(lambda x: x if x/10000%1==0 else 0)
raw['isNightAmount']=raw['notifytime'].apply(lambda x: 1 if int(x[11:13]) in (1,2,3,4) else 0)
raw['paymentchannel']=raw[pd.notnull(raw['paymentchannel'])]['paymentchannel'].apply(lambda x: 'DEBIT' if 'DEBIT' in x else ('CREDIT' if 'CREDIT' in x else x))

In [542]:
init=raw[raw['status']=='INIT']
success=raw[raw['status']=='SUCCESS']
failure=raw[raw['status']=='FAILURE']

## 近14天 发起交易

In [543]:
### 交易相关
# 交易笔数 金额 均值 标准差 最大值 最小值 中位数
initBasic=init.groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 云闪付信用卡交易笔数 金额 占比 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initCredit=init[init['cardtype']=='CREDIT'].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 微信支付宝信用卡交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
# notice init环节无微信支付宝支付方式信息
init=init
# 整数交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initIntAmt=init[(init['amount']!=0) & (init['isIntAmount']!=0)].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 大额交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initLargeAmt=init[init['amount']>499999].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 夜间交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initNightAmt=init[init['isNightAmount']==1].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 同额度交易笔数 金额
# todo 笔数占比 金额占比
initSameAmt=init.groupby(['merchantid','amount'])['amount'].agg(['count']).reset_index()
initSameAmt['amount']=initSameAmt.apply(lambda x: 0 if x['count']<4 else x['amount']*x['count'], axis=1)
initSameAmt['count']=initSameAmt.apply(lambda x: 0 if x['amount']==0 else x['count'],axis=1)
initSameAmt=initSameAmt.groupby(['merchantid']).sum()
# 交易频繁程度（平均间隔时间）
# todo
# 交易失败率 金额占比
failRate=failure.groupby(['merchantid','aa.date'])['amount'].agg(['count','sum']).groupby(['merchantid']).agg(['mean','max'])
failRate.columns=['count_mean','count_max','sum_mean','sum_max']


### 卡相关
# 单卡单日交易笔数最大值 金额最大值
# todo 笔数占比 金额占比
initSingleCard=init.groupby(['merchantid','aa.date','bankcardnum'])['amount'].agg(['count','sum']).groupby(['merchantid']).max()
# 卡号交易比（卡数量比交易量）
# todo 占比
initCard=init[pd.notnull(init['bankcardnum'])].groupby(['merchantid'])['bankcardnum'].nunique()


### 用户相关
# 单用户交易笔数 占比 最大值 金额 占比 最大值
# 用户交易比（用户数量比交易量）
# notice init环节无微信支付宝支付用户信息

## 近14天 成功交易

In [544]:
### 交易相关
# 交易笔数 金额 均值 标准差 最大值 最小值 中位数
successBasic=success.groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 云闪付信用卡交易笔数 金额 占比 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successCredit=success[success['cardtype']=='CREDIT'].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 微信信用卡交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
# notice init环节无微信支付宝支付方式信息
successWeCredit=success[(success['channel']=='UP_WE_CHAT')&(success['paymentchannel']=='CREDIT')].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 整数交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successIntAmt=success[(success['amount']!=0) & (success['isIntAmount']!=0)].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 大额交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successLargeAmt=success[success['amount']>499999].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 夜间交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successNightAmt=success[success['isNightAmount']==1].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 同额度交易笔数 金额
# todo 笔数占比 金额占比
successSameAmt=success.groupby(['merchantid','amount'])['amount'].agg(['count']).reset_index()
successSameAmt['amount']=successSameAmt.apply(lambda x: 0 if x['count']<4 else x['amount']*x['count'], axis=1)
successSameAmt['count']=successSameAmt.apply(lambda x: 0 if x['amount']==0 else x['count'],axis=1)
successSameAmt=successSameAmt.groupby(['merchantid']).sum()
# 交易频繁程度（平均间隔时间）
# todo


### 卡相关
# 单卡单日交易笔数最大值 金额最大值
# todo 笔数占比 金额占比
successSingleCard=success.groupby(['merchantid','aa.date','bankcardnum'])['amount'].agg(['count','sum']).groupby(['merchantid']).max()
# 卡号交易比（卡数量比交易量）
# todo 占比
successCard=success[pd.notnull(success['bankcardnum'])].groupby(['merchantid'])['bankcardnum'].nunique()


### 用户相关
# 单用户单日交易笔数最大值 金额最大值
# todo 笔数占比 金额占比
successSignleUser=success.groupby(['merchantid','aa.date','paymentidentity'])['amount'].agg(['count','sum']).groupby(['merchantid']).max()
# 用户交易比（用户数量比交易量）
# notice init环节无微信支付宝支付用户信息
successUser=success[pd.notnull(success['paymentidentity'])].groupby(['merchantid'])['bankcardnum'].nunique()

## 近14天汇总表

In [545]:
base14=initBasic.merge(initCredit,how='left',on=['merchantid'],suffixes=('_initBaisc','_initCredit')).\
                merge(initIntAmt,how='left',on=['merchantid'],suffixes=('','_initIntAmt')).\
                merge(initLargeAmt,how='left',on=['merchantid'],suffixes=('','_initLargeAmt')).\
                merge(initNightAmt,how='left',on=['merchantid'],suffixes=('','_initNightAmt')).\
                merge(initSameAmt,how='left',on=['merchantid'],suffixes=('','_initSameAmt')).\
                merge(failRate,how='left',on=['merchantid'],suffixes=('','_failRate')).\
                merge(initSingleCard,how='left',on=['merchantid'],suffixes=('','_initSingleCard')).\
                merge(initCard,how='left',on=['merchantid'],suffixes=('','_initCard')).\
                merge(successBasic,how='left',on=['merchantid'],suffixes=('','_successBasic')).\
                merge(successCredit,how='left',on=['merchantid'],suffixes=('','_successCredit')).\
                merge(successWeCredit,how='left',on=['merchantid'],suffixes=('','_successWeCredit')).\
                merge(successIntAmt,how='left',on=['merchantid'],suffixes=('','_successIntAmt')).\
                merge(successLargeAmt,how='left',on=['merchantid'],suffixes=('','_successLargeAmt')).\
                merge(successNightAmt,how='left',on=['merchantid'],suffixes=('','_successNightAmt')).\
                merge(successSameAmt,how='left',on=['merchantid'],suffixes=('','_successSameAmt')).\
                merge(successSingleCard,how='left',on=['merchantid'],suffixes=('','_successSingleCard')).\
                merge(successCard,how='left',on=['merchantid'],suffixes=('','_successCard'))

In [546]:
init=raw[(raw['status']=='INIT')&(raw['notifytime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))>datetime(2020, 4, 1, 0, 0, 0))]
success=raw[(raw['status']=='SUCCESS')&(raw['notifytime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))>datetime(2020, 4, 1, 0, 0, 0))]
failure=raw[(raw['status']=='FAILURE')&(raw['notifytime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))>datetime(2020, 4, 1, 0, 0, 0))]

## 近7天 发起交易

In [547]:
### 交易相关
# 交易笔数 金额 均值 标准差 最大值 最小值 中位数
initBasic=init.groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 云闪付信用卡交易笔数 金额 占比 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initCredit=init[init['cardtype']=='CREDIT'].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 微信支付宝信用卡交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
# notice init环节无微信支付宝支付方式信息
init=init
# 整数交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initIntAmt=init[(init['amount']!=0) & (init['isIntAmount']!=0)].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 大额交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initLargeAmt=init[init['amount']>499999].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 夜间交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
initNightAmt=init[init['isNightAmount']==1].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 同额度交易笔数 金额
# todo 笔数占比 金额占比
initSameAmt=init.groupby(['merchantid','amount'])['amount'].agg(['count']).reset_index()
initSameAmt['amount']=initSameAmt.apply(lambda x: 0 if x['count']<4 else x['amount']*x['count'], axis=1)
initSameAmt['count']=initSameAmt.apply(lambda x: 0 if x['amount']==0 else x['count'],axis=1)
initSameAmt=initSameAmt.groupby(['merchantid']).sum()
# 交易频繁程度（平均间隔时间）
# todo
# 交易失败率 金额占比
failRate=failure.groupby(['merchantid','aa.date'])['amount'].agg(['count','sum']).groupby(['merchantid']).agg(['mean','max'])
failRate.columns=['count_mean','count_max','sum_mean','sum_max']


### 卡相关
# 单卡单日交易笔数最大值 金额最大值
# todo 笔数占比 金额占比
initSingleCard=init.groupby(['merchantid','aa.date','bankcardnum'])['amount'].agg(['count','sum']).groupby(['merchantid']).max()
# 卡号交易比（卡数量比交易量）
# todo 占比
initCard=init[pd.notnull(init['bankcardnum'])].groupby(['merchantid'])['bankcardnum'].nunique()


### 用户相关
# 单用户交易笔数 占比 最大值 金额 占比 最大值
# 用户交易比（用户数量比交易量）
# notice init环节无微信支付宝支付用户信息

## 近7天 成功交易

In [548]:
### 交易相关
# 交易笔数 金额 均值 标准差 最大值 最小值 中位数
successBasic=success.groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 云闪付信用卡交易笔数 金额 占比 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successCredit=success[success['cardtype']=='CREDIT'].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 微信信用卡交易笔数 占比 金额 占比 均值 标准差 最大值 最小值 中位数
# notice init环节无微信支付宝支付方式信息
successWeCredit=success[(success['channel']=='UP_WE_CHAT')&(success['paymentchannel']=='CREDIT')].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 整数交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successIntAmt=success[(success['amount']!=0) & (success['isIntAmount']!=0)].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 大额交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successLargeAmt=success[success['amount']>499999].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 夜间交易笔数 金额 均值 标准差 最大值 最小值 中位数
# todo 笔数占比 金额占比
successNightAmt=success[success['isNightAmount']==1].groupby(['merchantid'])['amount'].agg(['count','sum','mean','std','max','min','median'])
# 同额度交易笔数 金额
# todo 笔数占比 金额占比
successSameAmt=success.groupby(['merchantid','amount'])['amount'].agg(['count']).reset_index()
successSameAmt['amount']=successSameAmt.apply(lambda x: 0 if x['count']<4 else x['amount']*x['count'], axis=1)
successSameAmt['count']=successSameAmt.apply(lambda x: 0 if x['amount']==0 else x['count'],axis=1)
successSameAmt=successSameAmt.groupby(['merchantid']).sum()
# 交易频繁程度（平均间隔时间）
# todo


### 卡相关
# 单卡单日交易笔数最大值 金额最大值
# todo 笔数占比 金额占比
successSingleCard=success.groupby(['merchantid','aa.date','bankcardnum'])['amount'].agg(['count','sum']).groupby(['merchantid']).max()
# 卡号交易比（卡数量比交易量）
# todo 占比
successCard=success[pd.notnull(success['bankcardnum'])].groupby(['merchantid'])['bankcardnum'].nunique()


### 用户相关
# 单用户单日交易笔数最大值 金额最大值
# todo 笔数占比 金额占比
successSignleUser=success.groupby(['merchantid','aa.date','paymentidentity'])['amount'].agg(['count','sum']).groupby(['merchantid']).max()
# 用户交易比（用户数量比交易量）
# notice init环节无微信支付宝支付用户信息
successUser=success[pd.notnull(success['paymentidentity'])].groupby(['merchantid'])['bankcardnum'].nunique()

## 近7天汇总表

In [549]:
base7=initBasic.merge(initCredit,how='left',on=['merchantid'],suffixes=('_initBaisc','_initCredit')).\
                merge(initIntAmt,how='left',on=['merchantid'],suffixes=('','_initIntAmt')).\
                merge(initLargeAmt,how='left',on=['merchantid'],suffixes=('','_initLargeAmt')).\
                merge(initNightAmt,how='left',on=['merchantid'],suffixes=('','_initNightAmt')).\
                merge(initSameAmt,how='left',on=['merchantid'],suffixes=('','_initSameAmt')).\
                merge(failRate,how='left',on=['merchantid'],suffixes=('','_failRate')).\
                merge(initSingleCard,how='left',on=['merchantid'],suffixes=('','_initSingleCard')).\
                merge(initCard,how='left',on=['merchantid'],suffixes=('','_initCard')).\
                merge(successBasic,how='left',on=['merchantid'],suffixes=('','_successBasic')).\
                merge(successCredit,how='left',on=['merchantid'],suffixes=('','_successCredit')).\
                merge(successWeCredit,how='left',on=['merchantid'],suffixes=('','_successWeCredit')).\
                merge(successIntAmt,how='left',on=['merchantid'],suffixes=('','_successIntAmt')).\
                merge(successLargeAmt,how='left',on=['merchantid'],suffixes=('','_successLargeAmt')).\
                merge(successNightAmt,how='left',on=['merchantid'],suffixes=('','_successNightAmt')).\
                merge(successSameAmt,how='left',on=['merchantid'],suffixes=('','_successSameAmt')).\
                merge(successSingleCard,how='left',on=['merchantid'],suffixes=('','_successSingleCard')).\
                merge(successCard,how='left',on=['merchantid'],suffixes=('','_successCard'))

In [554]:
base=pd.merge(base14,base7,how='left',on=['merchantid'],suffixes=('14','7'))

In [595]:
base.head(20)

,count_initBaisc14,sum_initBaisc14,mean_initBaisc14,std_initBaisc14,max_initBaisc14,min_initBaisc14,median_initBaisc14,count_initCredit14,sum_initCredit14,mean_initCredit14,...,mean_successNightAmt7,std_successNightAmt7,max_successNightAmt7,min_successNightAmt7,median_successNightAmt7,amount_successSameAmt7,count_successSameAmt7,count_successSingleCard7,sum_successSingleCard7,bankcardnum_successCard7
merchantid,,,,,,,,,,,,,,,,,,,,,
869100001010002,1,100,1.000000e+02,NaN,100,100,100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001010003,33,605250,1.834091e+04,2.210531e+04,51200,0,5000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001010004,93,1079400,1.160645e+04,3.142003e+04,99800,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6.0,546700.0,8.0
869100001010007,1,0,0.000000e+00,NaN,0,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001020001,6,0,0.000000e+00,0.000000e+00,0,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001020003,1,10,1.000000e+01,NaN,10,10,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001020004,21,807250,3.844048e+04,1.077861e+05,500000,0,300.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001030001,3,0,0.000000e+00,0.000000e+00,0,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869100001030002,72,696900,9.679167e+03,2.755091e+04,99200,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3000.0,10.0,2.0,86600.0,5.0


In [596]:
from sklearn.preprocessing import StandardScaler

In [632]:
base1=pd.DataFrame(StandardScaler().fit_transform(base))
base2=base1.fillna(-1)
base2.index=base.index
base2=base2.merge(risk,how='left',on=['merchantid'])
base2['label']=base2['merchantname'].apply(lambda x: 1 if pd.notnull(x) else 0)
base2.drop(['merchantname'],axis=1,inplace=True)

In [656]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [767]:
from sklearn.linear_model import LogisticRegression
trainx,testx,trainy,testy=train_test_split(base2.iloc[:,1:183],base2['label'],test_size=0.5, random_state=983)

lr=LogisticRegression(random_state=983)
lr.fit(trainx,trainy)
pred=lr.predict(testx)
pred_prob=lr.predict_proba(testx)[:,1]

cm=metrics.confusion_matrix(testy, pred_prob>0.5)
print('**Precision is:',(cm[0][0]+cm[1][1])/(sum(cm[0])+sum(cm[1])))
print('\n**Confusion matrix is:\n',cm)
print('\n**Classification report is:\n',metrics.classification_report(testy, pred))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Precision is: 0.97002457002457

**Confusion matrix is:
 [[3938   12]
 [ 110   10]]

**Classification report is:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3950
           1       0.45      0.08      0.14       120

    accuracy                           0.97      4070
   macro avg       0.71      0.54      0.56      4070
weighted avg       0.96      0.97      0.96      4070



In [775]:
from sklearn.ensemble import RandomForestRegressor
trainx,testx,trainy,testy=train_test_split(base2.iloc[:,1:183],base2['label'],test_size=0.5, random_state=983)

rf=RandomForestRegressor(n_estimators=10, max_depth=15, random_state=983)
rf.fit(trainx,trainy)
pred=rf.predict(testx)
# pred_prob=rf.predict_proba(testx)[:,1]

cm=metrics.confusion_matrix(testy, pred_prob>0.5)
print('**Precision is:',(cm[0][0]+cm[1][1])/(sum(cm[0])+sum(cm[1])))
print('\n**Confusion matrix is:\n',cm)
print('\n**Classification report is:\n',metrics.classification_report(testy, pred_prob>0.5))

**Precision is: 0.97002457002457

**Confusion matrix is:
 [[3938   12]
 [ 110   10]]

**Classification report is:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3950
           1       0.45      0.08      0.14       120

    accuracy                           0.97      4070
   macro avg       0.71      0.54      0.56      4070
weighted avg       0.96      0.97      0.96      4070



In [92]:
ali['alidate'].max(),ali['alidate'].min(),we['wedate'].max(),we['wedate'].min()

('2020/4/7', '2020/1/14', '2020/4/8 11:18', '2020/2/14 10:03')

In [169]:
raw.columns

Index(['status', 'notifytime', 'amount', 'client', 'merchantid',
       'merchantname', 'channel', 'settlementperiod', 'scantype', 'cardtype',
       'qrcode', 'opentime', 'merchanttype', 'paymentidentity', 'bankcardnum',
       'paymentchannel', 'terminalid', 'codeoffail', 'causeoffail', 'aa.date',
       'alimid', 'alidate', 'alitype', 'alidecription', 'wemname', 'wedate',
       'wemethod', 'wetype', 'wedescription'],
      dtype='object')

In [194]:
raw.groupby(['merchantid','status'],as_index=False).size().reset_index()

,merchantid,status,0
0,869100001010001,SUCCESS,5
1,869100001010002,INIT,1
2,869100001010002,SUCCESS,1
3,869100001010003,INIT,33
4,869100001010003,SUCCESS,12
5,869100001010004,INIT,93
6,869100001010004,SUCCESS,79
7,869100001010007,INIT,1
8,869100001020001,INIT,6
9,869100001020001,SUCCESS,1


In [619]:
raw=pd.merge(raw,ali,how='left',left_on=['merchantid'],right_on=['alimid'])
raw=pd.merge(raw,we,how='left',left_on=['merchantname'],right_on=['wemname'])

risk=raw[pd.notnull(raw['alitype'])][['merchantid','merchantname']]\
.append(raw[pd.notnull(raw['wetype'])][['merchantid','merchantname']], ignore_index=True)\
.drop_duplicates()\
.reset_index(drop=True)

# # 仅保留风控商户
# raw[pd.notnull(raw['alitype'])][['merchantid','merchantname']]\
# .append(raw[pd.notnull(raw['wetype'])][['merchantid','merchantname']], ignore_index=True)\
# .drop_duplicates()\
# .reset_index(drop=True)\
# .to_csv('riskmerchant0426.csv',index=False)